In [3]:
import os
from pathlib import Path
import numpy as np
from skimage import io, img_as_ubyte, morphology
from skimage.measure import label, regionprops
from concurrent.futures import ThreadPoolExecutor
# from configs import *
from configs import crop_species, weed_species, train_annotations_folder, train_images_folder, class_names
# import matplotlib.pyplot as plt

synthetic_individuals_crop_folder = '../WE3DS_DATASET/Synthetic/patches_new/crop'
synthetic_individuals_weed_folder = '../WE3DS_DATASET/Synthetic/patches_new/weed'

def create_directories():
    os.makedirs(synthetic_individuals_crop_folder, exist_ok=True)
    os.makedirs(synthetic_individuals_weed_folder, exist_ok=True)
    for crop in crop_species:
        os.makedirs(os.path.join(synthetic_individuals_crop_folder, crop.replace(' ', '_')), exist_ok=True)
    for weed in weed_species:
        os.makedirs(os.path.join(synthetic_individuals_weed_folder, weed.replace(' ', '_')), exist_ok=True)


def save_cropped_image(image, mask, current_mask, image_name, class_name, output_dir, idx, min_area=100):
    image_dir = os.path.join(output_dir, class_name, 'image')
    mask_dir = os.path.join(output_dir, class_name, 'mask')
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)
    
    # 保留原始的掩码值，不做二值化
    labeled_mask = label(current_mask)
    regions = regionprops(labeled_mask)
    
    image_height, image_width = image.shape[:2]
    
    for i, region in enumerate(regions):
        if region.area >= min_area:
            minr, minc, maxr, maxc = region.bbox
            if minr == 0 or minc == 0 or maxr == image_height or maxc == image_width:
                continue
            cropped_image = image[minr:maxr, minc:maxc, :]
            cropped_mask = mask[minr:maxr, minc:maxc]  # 保留裁切后的原始掩码
            
            file_name = f"{class_name}_{image_name[:-4]}_{idx}_{i}.png"
            io.imsave(os.path.join(image_dir, file_name), img_as_ubyte(cropped_image))
            io.imsave(os.path.join(mask_dir, file_name), cropped_mask)  # 保存原始掩码值


def process_image(item):
    name = item.name
    mask = io.imread(item)
    img = io.imread(img_path / name)

    for class_id, class_name in enumerate(class_names):
        if class_name == 'void':
            continue
        current_mask = (mask == class_id).astype(np.uint8)
        if class_name in crop_species:
            save_cropped_image(img, mask, current_mask, name, class_name, synthetic_individuals_crop_folder, class_id)
        elif class_name in weed_species:
            save_cropped_image(img, mask, current_mask, name, class_name, synthetic_individuals_weed_folder, class_id)

In [4]:
create_directories()

anno_path = Path(train_annotations_folder)
img_path = Path(train_images_folder)

with ThreadPoolExecutor() as executor:
    list(executor.map(process_image, anno_path.glob('*.png')))

print("Individual crop and weed images have been extracted and saved.")

/tmp/ipykernel_1707630/1365160636.py:45: UserWarning: ../WE3DS_DATASET/Synthetic/patches_new/crop/pea/mask/pea_img_00041_6_0.png is a low contrast image
  io.imsave(os.path.join(mask_dir, file_name), cropped_mask)  # 保存原始掩码值
/tmp/ipykernel_1707630/1365160636.py:45: UserWarning: ../WE3DS_DATASET/Synthetic/patches_new/crop/pea/mask/pea_img_00037_6_0.png is a low contrast image
  io.imsave(os.path.join(mask_dir, file_name), cropped_mask)  # 保存原始掩码值
/tmp/ipykernel_1707630/1365160636.py:45: UserWarning: ../WE3DS_DATASET/Synthetic/patches_new/crop/pea/mask/pea_img_00031_6_3.png is a low contrast image
  io.imsave(os.path.join(mask_dir, file_name), cropped_mask)  # 保存原始掩码值
/tmp/ipykernel_1707630/1365160636.py:45: UserWarning: ../WE3DS_DATASET/Synthetic/patches_new/crop/pea/mask/pea_img_00037_6_1.png is a low contrast image
  io.imsave(os.path.join(mask_dir, file_name), cropped_mask)  # 保存原始掩码值
/tmp/ipykernel_1707630/1365160636.py:45: UserWarning: ../WE3DS_DATASET/Synthetic/patches_new/crop/pe

Individual crop and weed images have been extracted and saved.


In [ ]:
import os
from pathlib import Path
import numpy as np
import imageio.v3 as iio

import matplotlib.pyplot as plt

img_path = '../WE3DS_DATASET/Synthetic/soil'
img_p = Path(img_path)

pure_soil_path = './datasets/WE3DS/pure_soil'
os.makedirs(pure_soil_path, exist_ok=True)


for item in img_p.glob('*.png'):
    name = item.name

    img = iio.imread(item)
    mask = iio.imread(item, mode='F')
    # 将掩码二值化
    binary_mask = (mask > 0).astype(np.uint8)
    max_area, best_rectangle = maximal_rectangle_area(binary_mask)

    # 可视化结果
    (top, left, bottom, right) = best_rectangle

    iio.imwrite(f'{pure_soil_path}/{name}', img[top:bottom, left:right, :])